In [1]:
import tweepy
import numpy as np
import pandas as pd
import re

In [2]:
with open("api_key.txt") as consumer_keys :
    consumer_key = consumer_keys.readline().strip()
    consumer_secret = consumer_keys.readline().strip()
    
with open("access_tokens.txt") as access_tokens :
    access_token = access_tokens.readline().strip()
    access_token_secret = access_tokens.readline().strip()

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [3]:
# get the latitude and longitude for all US cities 
# credit - https://simplemaps.com/data/us-cities
lat_long = pd.read_csv('uscities.csv', usecols=['lat','lng'])

In [150]:
list_of_tweets = []
latitude = []
longitude = []
for index, row in lat_long.iterrows() :
    lat = str(row['lat'])
    long = str(row['lng'])
    tweet_list = api.search(q='-is:retweet -RT', 
                            geocode= lat + ',' + long + ',' + '25mi', 
                            lang='en', result_type='recent', count=100)
    latitude.extend([row['lat'] for i in range(len(tweet_list))])
    longitude.extend([row['lng'] for i in range(len(tweet_list))])
    list_of_tweets.append(tweet_list)

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]

In [20]:
flatten = lambda list_of_lists : [tweet for sublist in list_of_lists for tweet in sublist]

In [21]:
tweets = flatten(list_of_tweets)

NameError: name 'list_of_tweets' is not defined

In [31]:
text = []
for tweet in range(len(tweet_list)) :
    temp = re.sub("(\@\S*) | (https\://[a-z0-9A-Z\.\-_/]*)", "", tweet_list[tweet].text)
    temp = re.sub("#", " ", temp)
    temp = re.sub("(\t)|(\n)", " ", temp)
    text.append(re.sub("[^a-z0-9A-Z!\-\.\ ]", "", temp))

In [32]:
twitter_data = pd.DataFrame({'lat':latitude, 'long':longitude, 'text':text})
twitter_data.to_csv('twitter_data.csv', index=False)